# Working with canela.lpnc.LPNC object

In [2]:
import os
from pprint import pprint
import numpy as np
import canela.lpnc as lpnc


## 1. Just need a path to lpnc XYZ

In [3]:
import importlib.resources
import canela.data

# use Au20(TBBT)16 as example system
with importlib.resources.path(canela.data, 'au20_tbbt16.xyz') as p:
    xyz_path = p

print(os.path.basename(xyz_path))

au20_tbbt16.xyz


## 2. Initialize LPNC object using XYZ path

In [4]:
nc = lpnc.LPNC(xyz_path)
print(nc)

Au20C160H208S16


## LPNC object contains lots of useful info regarding the nanocluster structure:

In [5]:
# ase.Atoms object of nanocluster
print(nc.atoms)
print(f'Nanocluster contains {len(nc.atoms)} atoms.')


Atoms(symbols='C160H208Au20S16', pbc=False, tags=...)
Nanocluster contains 404 atoms.


In [6]:
# separate core and shell atoms objects
print(nc.core)
print(nc.shell)

print('Core atom indices are maintained in each object as tags.')
print(f'Core atom indices: {nc.core.get_tags()}.')
print(f'These map directly back to atoms object: {nc.atoms[nc.core.get_tags()] == nc.core}')


Atoms(symbols='Au7', pbc=False, tags=...)
Atoms(symbols='C160H208Au13S16', pbc=False, tags=...)
Core atom indices are maintained in each object as tags.
Core atom indices: [  3   4   6  11 206 208 211].
These map directly back to atoms object: True


## Atom IDs should tell you all the info you need to know about each atom in the nanocluster

In [7]:
# atom IDs that tell you core-shell info, motif type, core layer, element type, etc.
for i in [3, 10, 20, 30]:
    print(f'Atom {i} ({nc.atoms[i].symbol:>2}) ID = {nc.ids[i]}')

print()
help(lpnc.get_atom_ids)

Atom 3 (Au) ID = C_S_01_Au
Atom 10 (Au) ID = S_M_-8_Au
Atom 20 ( C) ID = S_R_xx_xx
Atom 30 ( C) ID = S_R_xx_xx

Help on function get_atom_ids in module canela.lpnc.structure:

get_atom_ids(atoms, cs_details=None, motifs=None, scale=1.1)
    encode structural type of each atom in LPNC
    - (C[core] | S[shell], ...
    - For core:
        - (C, S[surface of core] | B[bulk], int[core layer])
        - core layer = 0[center], 1[layer 1], ..., n[surface]
    - For shell:
        - (S, R[R group of ligand])
        - (S, M[metal] | S[sulfur] , int[motif type], E[end] | M[middle])
        - end = sulfur atom that terminates a motif (at least one bond to core)
    
    Returns:
    (np.ndarray): 1D array of atom id's ordered by atoms object indices



## Core shell info dict

In [8]:
print(nc.info.keys(), '\n')
print('Atom indices that are: in <core>, in <shell>, <sulfido> sulfurs, <bridge> sulfurs.')
print("Number of shell interactions with core <nshellint> (used in MGT's Thermodynamic Stability Model).")
print(f'\nCore CN avg: {nc.info["corecnavg"]}\n')
print(f'Just Core CN avg:\n(excludes core-shell bonds from CN) {nc.info["justcorecnavg"]}')

dict_keys(['core', 'shell', 'sulfido', 'bridge', 'nshellint', 'corecnavg', 'justcorecnavg']) 

Atom indices that are: in <core>, in <shell>, <sulfido> sulfurs, <bridge> sulfurs.
Number of shell interactions with core <nshellint> (used in MGT's Thermodynamic Stability Model).

Core CN avg: 4.571428571428571

Just Core CN avg:
(excludes core-shell bonds from CN) 3.4285714285714284


## Motif info

nc.motifs is a dict with:

* key = motif type (1 = monomer, 2 = dimer, 0 = bridge, -1 = sulfido, -8 = octameric ring)
* value = np array of atom indices involved in each motif; shape = (n_motifs x n_atoms_in_motif)

In [9]:
pprint(nc.motifs)
print()
help(lpnc.count_motifs)

{-8: array([[214, 207, 212, 210, 215, 204, 216,   0,  16,   1,  15,  10,  12,
          5,  14,   7]]),
 1: array([[ 19,   2,  17],
       [219, 205, 217]]),
 3: array([[218, 209, 213,   8,  13,   9,  18]])}

Help on function count_motifs in module canela.lpnc.structure:

count_motifs(atom, scale=1.1, show=False, sulfido=[])
    algorithmically determine motif types and counts of LPNC
    
    Arguments:
        atom (ase.Atoms): metal NC atoms object
    
    Keyword Arguments:
        full_cluster (bool): if False, atoms object only contains shell
                               (default: False)
        scale (float): scales covalent radii when calculating neighborlist
                         (default: 1.0)
        show (bool): if True, motif info is printed
                       (default: False)
        sulfido (list): list of sulfido atom indices found from
                          get_core_shell function
                          (default: [])
    
    Returns:
        motif inf

## Bonds object

`nc.bonds`

* coordination dict of atom connectivity: `nc.bonds.coord_dict`
* array of coordination numbers: `nc.bonds.cns`
* bond array (`nc.bonds.bond_arr`) and halfbonds array (`nc.bonds.halfbond_arr`)

In [10]:
print(nc.bonds, '\n')

# coord_dict of atom connectivity
for key in [10, 20, 30]:
    print(f'Atom {key} ({nc.atoms[key].symbol}) has a CN of {nc.bonds.cns[key]}.')
    print(f'\tIt is bonded to: {nc.bonds.coord_dict[key]}')



Atom 10 (Au) has a CN of 2.
	It is bonded to: [12, 15]
Atom 20 (C) has a CN of 3.
	It is bonded to: [14, 21, 28]
Atom 30 (C) has a CN of 3.
	It is bonded to: [15, 31, 38]


In [11]:
print('Half bonds array contains [0, 1] and [1, 0].')
print('Bonds array only contains [0, 1].')
print(f'Length of bonds ({len(nc.bonds.bond_arr)}) == 1/2 length of half_bonds ({len(nc.bonds.halfbond_arr)})')


Half bonds array contains [0, 1] and [1, 0].
Bonds array only contains [0, 1].
Length of bonds (430) == 1/2 length of half_bonds (860)


## Bond distance analysis: calculate bond distances based on atom ID type

`lpnc.get_bond_distance_analysis(nc.atoms)`

### Returns nested dict with atom ID keys and lists of bond distances (in Angstrom)

In [12]:
# currently takes an Atoms object or xyz path
# Next update: allow LPNC object as input
bond_analysis = lpnc.get_bond_distance_analysis(nc.atoms)

# bond_analysis is a nested dict that contains
# lists of bond distances between atom ID types
for atom_id in bond_analysis:
    if bond_analysis[atom_id]:
        print(f'\n{atom_id} bonded to:')
        for bonded2 in bond_analysis[atom_id]:
            # value is a list of distances
            # NOTE: this should be an array in the future
            #       for now, let's just cast it
            dists = np.array(bond_analysis[atom_id][bonded2])
            print(f'  {bonded2}: {dists.mean():.3f} +/- {dists.std():.3f}')


S_M_-8_Au bonded to:
  S_S_-8_xM: 2.371 +/- 0.016
  C_S_01_Au: 2.966 +/- 0.000

S_M_01_Au bonded to:
  S_S_01_xE: 2.353 +/- 0.003

C_S_01_Au bonded to:
  C_B_00_Au: 2.819 +/- 0.009
  S_S_01_xE: 2.449 +/- 0.015
  S_M_-8_Au: 2.966 +/- 0.000
  C_S_01_Au: 2.830 +/- 0.033
  S_S_03_xE: 2.412 +/- 0.000

C_B_00_Au bonded to:
  C_S_01_Au: 2.819 +/- 0.009

S_M_03_Au bonded to:
  S_S_03_xM: 2.357 +/- 0.005
  S_S_03_xE: 2.347 +/- 0.000

S_S_-8_xM bonded to:
  S_M_-8_Au: 2.371 +/- 0.016
  S_R_xx_xx: 1.793 +/- 0.002

S_S_03_xM bonded to:
  S_M_03_Au: 2.357 +/- 0.005
  S_R_xx_xx: 1.792 +/- 0.000

S_S_01_xE bonded to:
  S_M_01_Au: 2.353 +/- 0.003
  C_S_01_Au: 2.449 +/- 0.015
  S_R_xx_xx: 1.786 +/- 0.001

S_S_03_xE bonded to:
  C_S_01_Au: 2.412 +/- 0.000
  S_M_03_Au: 2.347 +/- 0.000
  S_R_xx_xx: 1.793 +/- 0.000
